In [70]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import utils

In [78]:
location = 'A'
x_train, x_train_estimated, x_test = utils.preprocess_category(location)
x_train.drop(["time"], axis=1, inplace=True)
x_train_estimated.drop(["time"], axis=1, inplace=True)

x_train['date_forecast'] = pd.to_datetime(x_train['date_forecast'])
x_train_estimated['date_forecast'] = pd.to_datetime(x_train_estimated['date_forecast'])

x_test.fillna(0, inplace=True)

x_train['ID'] = pd.factorize(x_train['date_forecast'].dt.date)[0]

last_id_x_train = x_train['ID'].max() + 1
x_train_estimated['ID'] = pd.factorize(x_train_estimated['date_forecast'].dt.date)[0] + last_id_x_train

x_test['ID'] = pd.factorize(x_test['date_forecast'].dt.date)[0] + last_id_x_train

x_train

Total data points: 34085
Data points to be removed: 0


/cluster/home/hansal/ml/utils.py:135: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_df = estimated_df.resample('H').mean()
/cluster/home/hansal/ml/utils.py:144: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_df = test_df.resample('H').mean()


absolute_humidity_2m:gm3  air_density_2m:kgm3  ceiling_height_agl:m  \
0                         7.700              1.22825           1728.949951   
1                         7.700              1.22350           1689.824951   
2                         7.875              1.21975           1563.224976   
3                         8.425              1.21800           1283.425049   
4                         8.950              1.21800           1003.500000   
...                         ...                  ...                   ...   
29662                     5.800              1.28050           1291.500000   
29663                     5.575              1.28400              0.000000   
29664                     5.350              1.28700              0.000000   
29665                     5.200              1.29050              0.000000   
29666                     5.000              1.29200              0.000000   

       clear_sky_energy_1h:J  clear_sky_rad:W  cloud_base_agl:m  \
0                   0.000000             0.00       1728.949951   
1                   0.000000             0.00       1689.824951   
2                   0.000000             0.00       1563.224976   
3                 208.649994             0.75       1283.425049   
4               32468.150391            23.10       1003.500000   
...                      ...              ...               ...   
29662               0.000000             0.00       1192.550049   
29663               0.000000             0.00        686.900024   
29664               0.000000             0.00        149.500000   
29665               0.000000             0.00          0.000000   
29666               0.000000             0.00          0.000000   

            date_forecast  dew_or_rime:idx  dew_point_2m:K  diffuse_rad:W  \
0     2019-06-02 22:00:00              0.0      280.299988          0.000   
1     2019-06-02 23:00:00              0.0      280.299988          0.000   
2     2019-06-03 00:00:00              0.0      280.649994          0.000   
3     2019-06-03 01:00:00              0.0      281.674988          0.300   
4     2019-06-03 02:00:00              0.0      282.500000         11.975   
...                   ...              ...             ...            ...   
29662 2022-10-20 20:00:00              0.0      275.799988          0.000   
29663 2022-10-20 21:00:00              0.0      275.200012          0.000   
29664 2022-10-20 22:00:00              0.0      274.600006          0.000   
29665 2022-10-20 23:00:00              0.0      274.174988          0.000   
29666 2022-10-21 00:00:00              0.0      273.675018          0.000   

       ...  sun_elevation:d  super_cooled_liquid_water:kgm2  t_1000hPa:K  \
0      ...        -3.774250                             0.0   286.225006   
1      ...        -4.357250                             0.0   286.899994   
2      ...        -3.309500                             0.0   286.950012   
3      ...        -0.822500                             0.0   286.750000   
4      ...         3.051250                             0.0   286.450012   
...    ...              ...                             ...          ...   
29662  ...       -29.704250                             0.0   278.250000   
29663  ...       -33.860748                             0.0   277.850006   
29664  ...       -36.270000                             0.0   277.475006   
29665  ...       -36.614998                             0.0   277.149994   
29666  ...       -35.368500                             0.0   276.700012   

       total_cloud_cover:p  visibility:m  wind_speed_10m:ms  \
0               100.000000  40386.476562              3.600   
1               100.000000  33770.648438              3.350   
2               100.000000  13595.500000              3.050   
3               100.000000   2321.850098              2.725   
4                99.224998  11634.799805              2.550   
...                    ...           ...         

In [79]:
train_data = TimeSeriesDataFrame.from_data_frame(
    x_train,
    id_column="ID",
    timestamp_column="date_forecast"
)

ts_lengths = train_data.num_timesteps_per_item()
items_to_keep = ts_lengths.index[ts_lengths >= 3]
train_data = train_data.query("item_id in @items_to_keep")

train_data = train_data.to_regular_index(freq="H")
print(f"Data has frequency '{train_data.freq}'")

Data has frequency 'H'


In [80]:
tuning_data = TimeSeriesDataFrame.from_data_frame(
    x_train_estimated,
    id_column="ID",
    timestamp_column="date_forecast"
)

thirty_percent_index = int(len(tuning_data) * 0.3)
tuning_data = tuning_data.iloc[:thirty_percent_index]

ts_lengths = tuning_data.num_timesteps_per_item()
items_to_keep = ts_lengths.index[ts_lengths >= 3]
tuning_data = tuning_data.query("item_id in @items_to_keep")

tuning_data = tuning_data.to_regular_index(freq="H")
print(f"Data has frequency '{tuning_data.freq}'")
tuning_data

Data has frequency 'H'


absolute_humidity_2m:gm3  air_density_2m:kgm3  \
item_id timestamp                                                            
1239    2022-10-29 00:00:00                     8.150              1.22975   
        2022-10-29 01:00:00                     8.200              1.22850   
        2022-10-29 02:00:00                     8.375              1.22700   
        2022-10-29 03:00:00                     8.550              1.22325   
        2022-10-29 04:00:00                     8.750              1.22100   
...                                               ...                  ...   
1293    2022-12-22 22:00:00                     2.900              1.30200   
        2022-12-22 23:00:00                     2.800              1.30300   
1294    2022-12-23 00:00:00                     2.700              1.30500   
        2022-12-23 01:00:00                     2.700              1.30500   
        2022-12-23 02:00:00                     2.700              1.30425   

                             ceiling_height_agl:m  clear_sky_energy_1h:J  \
item_id timestamp                                                          
1239    2022-10-29 00:00:00           1882.000000                    0.0   
        2022-10-29 01:00:00           1622.400024                    0.0   
        2022-10-29 02:00:00           1322.275024                    0.0   
        2022-10-29 03:00:00           1039.099976                    0.0   
        2022-10-29 04:00:00            854.599976                    0.0   
...                                           ...                    ...   
1293    2022-12-22 22:00:00             28.750000                    0.0   
        2022-12-22 23:00:00            253.649994                    0.0   
1294    2022-12-23 00:00:00           2049.574951                    0.0   
        2022-12-23 01:00:00           2572.500000                    0.0   
        2022-12-23 02:00:00           3869.899902                    0.0   

                             clear_sky_rad:W  cloud_base_agl:m  \
item_id timestamp                                                
1239    2022-10-29 00:00:00              0.0       1675.050049   
        2022-10-29 01:00:00              0.0       1547.550049   
        2022-10-29 02:00:00              0.0       1053.724976   
        2022-10-29 03:00:00              0.0        703.250000   
        2022-10-29 04:00:00              0.0        661.049988   
...                                      ...               ...   
1293    2022-12-22 22:00:00              0.0         39.099998   
        2022-12-22 23:00:00              0.0         38.224998   
1294    2022-12-23 00:00:00              0.0        377.450012   
        2022-12-23 01:00:00              0.0        153.800003   
        2022-12-23 02:00:00              0.0        662.200012   

                             dew_or_rime:idx  dew_point_2m:K  diffuse_rad:W  \
item_id timestamp                                                             
1239    2022-10-29 00:00:00             1.00      280.924988            0.0   
        2022-10-29 01:00:00             1.00      281.049988            0.0   
        2022-10-29 02:00:00             1.00      281.349976            0.0   
        2022-10-29 03:00:00             0.75      281.700012            0.0   
        2022-10-29 04:00:00             0.00      282.100006            0.0   
...                                      ...             ...            ...   
1293    2022-12-22 22:00:00             0.00      266.674988            0.0   
        2022-12-22 23:00:00             0.00      266.500000            0.0   
1294    2022-12-23 00:00:00             0.00      265.899994            0.0   
        2022-12-23 01:00:00             0.00      265.799988            0.0   
        2022-12-23 02:00:00             0.00      265.799988            0.0   

                             diffuse_rad_1h:J  ...  sun_azimuth:d  \
item_id timestamp                              ...                  


In [81]:
test_data = TimeSeriesDataFrame.from_data_frame(
    x_test,
    id_column="ID",
    timestamp_column="date_forecast"
)

ts_lengths = test_data.num_timesteps_per_item()
items_to_keep = ts_lengths.index[ts_lengths >= 3]
test_data = test_data.query("item_id in @items_to_keep")

test_data = test_data.to_regular_index(freq="H")
print(f"Data has frequency '{test_data.freq}'")
test_data

Data has frequency 'H'


absolute_humidity_2m:gm3  air_density_2m:kgm3  \
item_id timestamp                                                            
1238    2023-05-01 00:00:00                     4.325              1.28675   
        2023-05-01 01:00:00                     4.275              1.28600   
        2023-05-01 02:00:00                     4.150              1.28375   
        2023-05-01 03:00:00                     4.025              1.28200   
        2023-05-01 04:00:00                     3.900              1.28100   
...                                               ...                  ...   
1301    2023-07-03 19:00:00                     8.350              1.19725   
        2023-07-03 20:00:00                     8.525              1.20050   
        2023-07-03 21:00:00                     8.825              1.20450   
        2023-07-03 22:00:00                     9.025              1.20700   
        2023-07-03 23:00:00                     9.050              1.20775   

                             ceiling_height_agl:m  clear_sky_energy_1h:J  \
item_id timestamp                                                          
1238    2023-05-01 00:00:00            912.700012               0.000000   
        2023-05-01 01:00:00           1482.099976               0.000000   
        2023-05-01 02:00:00           1791.300049               0.000000   
        2023-05-01 03:00:00           2312.875000           10124.424805   
        2023-05-01 04:00:00           2198.299805          141748.593750   
...                                           ...                    ...   
1301    2023-07-03 19:00:00           3638.899902          477093.187500   
        2023-07-03 20:00:00           3552.100098          184335.671875   
        2023-07-03 21:00:00           2315.000000           37429.328125   
        2023-07-03 22:00:00           2202.800049             360.049988   
        2023-07-03 23:00:00           2015.675049               0.000000   

                             clear_sky_rad:W  cloud_base_agl:m  \
item_id timestamp                                                
1238    2023-05-01 00:00:00         0.000000       1061.550049   
        2023-05-01 01:00:00         0.000000       1075.100098   
        2023-05-01 02:00:00         0.000000       1200.400024   
        2023-05-01 03:00:00        11.675000       1179.849976   
        2023-05-01 04:00:00        76.875000        920.049988   
...                                      ...               ...   
1301    2023-07-03 19:00:00        85.099998       2013.750000   
        2023-07-03 20:00:00        24.799999       1610.900024   
        2023-07-03 21:00:00         1.275000       1622.800049   
        2023-07-03 22:00:00         0.000000       1767.550049   
        2023-07-03 23:00:00         0.000000       1438.449951   

                             dew_or_rime:idx  dew_point_2m:K  diffuse_rad:W  \
item_id timestamp                                                             
1238    2023-05-01 00:00:00              0.0      271.650024       0.000000   
        2023-05-01 01:00:00              0.0      271.450012       0.000000   
        2023-05-01 02:00:00              0.0      271.049988       0.000000   
        2023-05-01 03:00:00              0.0      270.649994       9.375000   
        2023-05-01 04:00:00              0.0      270.375000      47.400002   
...                                      ...             ...            ...   
1301    2023-07-03 19:00:00              0.0      281.575012      33.625000   
        2023-07-03 20:00:00              0.0      281.850006      14.325000   
        2023-07-03 21:00:00              0.0      282.325012       1.300000   
        2023-07-03 22:00:00              0.0      282.675018       0.000000   
        2023-07-03 23:00:00              0.0      282.675018       0.000000   

                             diffuse_rad_1h:J  ...  sun_azimuth:d  \
item_id timestamp                              ...                  


In [82]:
predictor = TimeSeriesPredictor(
    prediction_length=24,
    path="new_testing",
    target="pv_measurement",
    eval_metric="RMSE",
    known_covariates_names=train_data.drop('pv_measurement', axis=1).columns.tolist()
)

In [83]:
predictor.fit(
    train_data,
    presets="high_quality",
    time_limit=30,
    num_val_windows= 3,
    tuning_data=tuning_data
)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: high_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'high_quality',
 'num_val_windows': 3,
 'prediction_length': 24,
 'random_seed': None,
 'target': 'pv_measurement',
 'time_limit': 30,
 'verbosity': 2}
Provided training data set with 29664 rows, 1236 items (item = single time series). Average time series length is 24.0. Data frequency is 'H'.
Provided tuning data set with 1323 rows, 56 items. Average time series length is 23.6.
Multi-window backtesting is disabled (setting num_val_windows = 0)

It is recommended that all time series in train_data have length >= prediction_length + 1 (at least 25). Otherwise the predictor may not work as expected. 
Please reduce prediction_length or provide longer time series as train_data. 
AutoGluon will save models t

In [67]:
predictions = predictor.predict(train_data, known_covariates=test_data)
predictions

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 predictions = predictor.predict(train_data, known_covariates_names=test_data)                │
│   2 predictions                                                                                  │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: predict() got an unexpected keyword argument 'known_covariates_names'